In [1]:
import pandas

In [2]:
import cleanup_fragile_states as csf

In [3]:
csf.cleanup_fragile_states()

In [4]:
data_raw = pandas.read_csv("data/fragilestatesindex_joined.csv")

In [5]:
data_raw[0:10]

,Country Name,Demographic Pressures 2006,External Intervention 2006,Factionalized Elites 2006,Group Grievance 2006,Human Flight 2006,Human Rights 2006,Legitimacy of the State 2006,Poverty and Economic Decline 2006,Public Services 2006,...,Group Grievance 2014,Human Flight 2014,Human Rights 2014,Legitimacy of the State 2014,Poverty and Economic Decline 2014,Public Services 2014,Refugees and IDPs 2014,Security Apparatus 2014,Total 2014,Uneven Development 2014
0,Sudan,9.6,9.8,9.1,9.7,9.1,9.8,9.5,7.5,9.5,...,10.0,6.8,9.9,9.7,8.8,9.9,10.0,9.9,112.9,8.9
1,"Congo, D.R.",9.5,10.0,9.6,9.1,8.0,9.5,9.0,8.1,9.0,...,9.3,8.9,9.8,9.1,9.1,9.6,10.0,9.4,112.6,8.7
2,Guinea,7.5,8.5,9.0,8.1,8.4,8.1,9.1,8.0,9.0,...,8.1,8.3,8.6,9.3,8.3,8.8,8.4,8.1,102.8,8.3
3,Brazil,6.5,4.7,3.2,5.7,5.0,5.3,5.5,2.7,6.7,...,8.0,5.0,5.2,5.8,6.4,4.4,6.5,6.2,72.0,5.6
4,Romania,6.5,5.5,4.1,5.4,5.5,4.8,6.2,5.9,5.3,...,6.2,4.9,6.0,7.4,5.4,6.4,2.7,6.2,71.6,8.7
5,Bulgaria,6.0,5.5,4.1,4.0,6.0,4.9,6.2,4.3,5.3,...,5.6,6.8,4.6,4.1,6.8,7.0,5.8,4.6,71.5,8.4
6,Croatia,5.7,5.8,3.9,6.5,5.0,4.6,4.2,5.8,4.1,...,5.7,5.7,6.8,6.7,5.8,4.4,6.7,5.6,71.3,5.3
7,Kuwait,5.7,4.0,7.5,4.5,4.0,6.5,6.8,2.8,3.0,...,6.1,5.6,6.0,5.8,4.9,6.3,4.3,7.6,71.1,6.9
8,Ghana,5.5,4.5,3.5,5.0,8.0,4.4,5.5,4.0,6.8,...,6.2,8.8,4.1,5.6,6.7,6.3,4.1,5.5,71.0,6.5
9,Panama,6.5,4.0,3.0,4.5,5.0,4.8,4.9,5.7,5.8,...,4.8,6.0,7.0,6.8,4.8,4.9,5.0,6.0,70.8,5.9


In [15]:
mabc = pandas.read_csv('data/monthly_arrivals_by_country_filtered.csv')

In [16]:
mabc[0:10]

,country_en,value,month,month_en,date
0,Greece,955,1,January,2014-01
1,Greece,1001,2,February,2014-02
2,Greece,1501,3,March,2014-03
3,Greece,1257,4,April,2014-04
4,Greece,1703,5,May,2014-05
5,Greece,3198,6,June,2014-06
6,Greece,3927,7,July,2014-07
7,Greece,6742,8,August,2014-08
8,Greece,7454,9,September,2014-09
9,Greece,7432,10,October,2014-10


In [17]:
fsi = pandas.read_csv("data/fragilestatesindex_joined.csv")

In [29]:
fsi[0:15]

,Country Name,Demographic Pressures 2006,External Intervention 2006,Factionalized Elites 2006,Group Grievance 2006,Human Flight 2006,Human Rights 2006,Legitimacy of the State 2006,Poverty and Economic Decline 2006,Public Services 2006,...,Group Grievance 2014,Human Flight 2014,Human Rights 2014,Legitimacy of the State 2014,Poverty and Economic Decline 2014,Public Services 2014,Refugees and IDPs 2014,Security Apparatus 2014,Total 2014,Uneven Development 2014
0,Sudan,9.6,9.8,9.1,9.7,9.1,9.8,9.5,7.5,9.5,...,10.0,6.8,9.9,9.7,8.8,9.9,10.0,9.9,112.9,8.9
1,"Congo, D.R.",9.5,10.0,9.6,9.1,8.0,9.5,9.0,8.1,9.0,...,9.3,8.9,9.8,9.1,9.1,9.6,10.0,9.4,112.6,8.7
2,Guinea,7.5,8.5,9.0,8.1,8.4,8.1,9.1,8.0,9.0,...,8.1,8.3,8.6,9.3,8.3,8.8,8.4,8.1,102.8,8.3
3,Brazil,6.5,4.7,3.2,5.7,5.0,5.3,5.5,2.7,6.7,...,8.0,5.0,5.2,5.8,6.4,4.4,6.5,6.2,72.0,5.6
4,Romania,6.5,5.5,4.1,5.4,5.5,4.8,6.2,5.9,5.3,...,6.2,4.9,6.0,7.4,5.4,6.4,2.7,6.2,71.6,8.7
5,Bulgaria,6.0,5.5,4.1,4.0,6.0,4.9,6.2,4.3,5.3,...,5.6,6.8,4.6,4.1,6.8,7.0,5.8,4.6,71.5,8.4
6,Croatia,5.7,5.8,3.9,6.5,5.0,4.6,4.2,5.8,4.1,...,5.7,5.7,6.8,6.7,5.8,4.4,6.7,5.6,71.3,5.3
7,Kuwait,5.7,4.0,7.5,4.5,4.0,6.5,6.8,2.8,3.0,...,6.1,5.6,6.0,5.8,4.9,6.3,4.3,7.6,71.1,6.9
8,Ghana,5.5,4.5,3.5,5.0,8.0,4.4,5.5,4.0,6.8,...,6.2,8.8,4.1,5.6,6.7,6.3,4.1,5.5,71.0,6.5
9,Panama,6.5,4.0,3.0,4.5,5.0,4.8,4.9,5.7,5.8,...,4.8,6.0,7.0,6.8,4.8,4.9,5.0,6.0,70.8,5.9
